In [47]:
import time
from io import StringIO

import pandas as pd
import requests
import duckdb

In [2]:
# geodaten via https://opendata.swiss/de/dataset/geodaten-zu-den-eidgenoessischen-abstimmungsvorlagen
topojson = requests.get('https://dam-api.bfs.admin.ch/hub/api/dam/assets/22344268/master').json()
# nur gemeinden im kanton bern behalten
topojson['objects']['K4voge_20220501_gf']['geometries'] = [g for g in topojson['objects']['K4voge_20220501_gf']['geometries'] if g['properties']['kantName'] == 'Bern']
be_gemeinde_nummern = dict([(g['properties']['vogeId'], g['properties']['vogeName']) for g in topojson['objects']['K4voge_20220501_gf']['geometries']])

try:
    result_df = pd.read_csv('gemeinde_resultate_listen.csv', encoding='utf-8')
except FileNotFoundError:
    for g in be_gemeinde_nummern.keys():
        try:
            resp = requests.get(f'https://www.bewas.sites.be.ch/2022/2022-03-27/WAHL_GROSSRAT/reportCsvGemeindeResultat-{g}.csv')
        except requests.ConnectionError as e:
            print('failed to download', g, be_gemeinde_nummern[g], f'https://www.bewas.sites.be.ch/2022/2022-03-27/WAHL_GROSSRAT/reportCsvGemeindeResultat-{g}.csv', e)
            continue
        time.sleep(0.5)
        data = resp.content.decode('windows-1252')
        lines = data.splitlines()
        lines = lines[5:[i for i in range(len(lines)) if lines[i].startswith('Total;')][0]]
        lines.insert(0, 'Listennummer;Partei;Stimmen;Stimmen-Prozent')
        df = pd.read_csv(StringIO('\n'.join(lines)), sep=';', converters={'Stimmen': lambda s: int(s.replace('\'', '')), 'Stimmen-Prozent': lambda s: float(s.replace('%', ''))})
        df['Gemeinde-Nummer'] = g
        df['Gemeinde-Name'] = be_gemeinde_nummern[g]
        result_df = df if result_df is None else pd.concat([result_df, df])
    result_df.to_csv('gemeinde_resultate_listen.csv', index=False, encoding='utf-8')

downloading data for 301 Aarberg
downloading data for 302 Bargen (BE)
downloading data for 303 Grossaffoltern
downloading data for 304 Kallnach
downloading data for 305 Kappelen
downloading data for 306 Lyss
downloading data for 307 Meikirch
downloading data for 309 Radelfingen
downloading data for 310 Rapperswil (BE)
downloading data for 311 Schüpfen
downloading data for 312 Seedorf (BE)
downloading data for 321 Aarwangen
downloading data for 322 Auswil
downloading data for 323 Bannwil
downloading data for 324 Bleienbach
downloading data for 325 Busswil bei Melchnau
downloading data for 326 Gondiswil
downloading data for 329 Langenthal
downloading data for 331 Lotzwil
downloading data for 332 Madiswil
downloading data for 333 Melchnau
downloading data for 335 Oeschenbach
downloading data for 336 Reisiswil
downloading data for 337 Roggwil (BE)
downloading data for 338 Rohrbach
downloading data for 339 Rohrbachgraben
downloading data for 340 Rütschelen
downloading data for 341 Schwarzhä

In [84]:
duckdb.query('''
    select "Gemeinde-Nummer", "Gemeinde-Name", sum(Stimmen) "Stimmen", sum("Stimmen-Prozent") "Stimmen-Prozent"
    from result_df
    where "Partei" in ('VERTS', 'JV', 'PVA', 'JGGL / JVVC', 'JGGG / JVVS', 'Grüne', 'JGOA', 'GRÜNE', 'JG', 'Grüne', 'GFL', 'JA!', 'GB')
    group by 1, 2
    having sum("Stimmen-Prozent") >= 10
    order by 3 desc
    ''').to_df()

,Gemeinde-Nummer,Gemeinde-Name,Stimmen,Stimmen-Prozent
0,351,Bern,165870.0,25.489991
1,371,Biel/Bienne,34521.0,16.975899
2,355,Köniz,32513.0,17.728304
3,942,Thun,24347.0,16.751525
4,356,Muri bei Bern,11018.0,14.017812
...,...,...,...,...
95,422,Rüti bei Lyssach,64.0,10.174881
96,591,Saxeten,48.0,11.822660
97,442,Romont (BE),42.0,11.052632
98,708,Schelten,30.0,31.250000


In [85]:
duckdb.query('''
    select "Gemeinde-Nummer", "Gemeinde-Name", sum("Stimmen") "Stimmen",
        sum(case when "Partei" in ('VERTS', 'JV', 'PVA', 'JGGL / JVVC', 'JGGG / JVVS', 'Grüne', 'JGOA', 'GRÜNE', 'JG', 'Grüne', 'GFL', 'JA!', 'GB') then "Stimmen" else 0 end) "Grüne-Stimmen",
        sum(case when "Partei" in ('VERTS', 'JV', 'PVA', 'JGGL / JVVC', 'JGGG / JVVS', 'Grüne', 'JGOA', 'GRÜNE', 'JG', 'Grüne', 'GFL', 'JA!', 'GB') then "Stimmen-Prozent" else 0 end) "Grüne-Stimmen-Prozent"
    from result_df
    group by 1, 2
    having sum("Stimmen") > 15000 or sum(case when "Partei" in ('VERTS', 'JV', 'PVA', 'JGGL / JVVC', 'JGGG / JVVS', 'Grüne', 'JGOA', 'GRÜNE', 'JG', 'Grüne', 'GFL', 'JA!', 'GB') then "Stimmen-Prozent" else 0 end) > 12
    order by 3 desc
    ''').to_df().to_csv('grossratswahlen 2022 gruene gemeinden.csv', encoding='utf-8')